This notebook shows how to pull the data from the crime statistics database using Socrata.  
<br>
The pulled database is then used as input for the modeling files (notebooks named "Modeling"). 


# Getting the Data

In [ ]:
import urllib.request
import pandas as pd
from sodapy import Socrata

In [ ]:
client = Socrata("data.sfgov.org", None)

In [ ]:
results = client.get("wg3w-h783", limit = 2000000)

In [ ]:
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,:@computed_region_qgnn_b9vv,:@computed_region_26cr_cadq,:@computed_region_ajp5_b2md,:@computed_region_6pnf_4xz7,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,filed_online,:@computed_region_2dwj_jsy4,:@computed_region_jg9y_a9du,:@computed_region_y6ts_4iup
0,2018-01-01T09:26:00.000,2018-01-01T00:00:00.000,09:26,2018,Monday,2018-01-01T09:27:00.000,61893007041,618930,171052174,173641140,...,2,9,1,2,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01T02:30:00.000,2018-01-01T00:00:00.000,02:30,2018,Monday,2018-01-01T08:21:00.000,61893105041,618931,180000768,180010668,...,9,1,7,2,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01T10:00:00.000,2018-01-01T00:00:00.000,10:00,2018,Monday,2018-01-01T10:20:00.000,61893275000,618932,180000605,180010893,...,4,10,36,2,1,NaN,NaN,NaN,NaN,NaN
3,2018-01-01T10:03:00.000,2018-01-01T00:00:00.000,10:03,2018,Monday,2018-01-01T10:04:00.000,61893565015,618935,180000887,180011579,...,9,1,28,1,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01T09:01:00.000,2018-01-01T00:00:00.000,09:01,2018,Monday,2018-01-01T09:39:00.000,61893607041,618936,171052958,180011403,...,6,3,6,2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445159,2021-03-29T18:30:00.000,2021-03-29T00:00:00.000,18:30,2021,Monday,2021-03-29T23:44:00.000,101847606244,1018476,216035027,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
445160,2021-04-03T13:25:00.000,2021-04-03T00:00:00.000,13:25,2021,Saturday,2021-04-03T16:30:00.000,101860606244,1018606,210206856,210932079,...,6,6,32,1,NaN,NaN,NaN,NaN,NaN,NaN
445161,2021-04-03T13:58:00.000,2021-04-03T00:00:00.000,13:58,2021,Saturday,2021-04-03T14:11:00.000,101861304014,1018613,210206492,210931580,...,6,3,32,1,NaN,NaN,NaN,NaN,NaN,NaN
445162,2021-04-03T16:30:00.000,2021-04-03T00:00:00.000,16:30,2021,Saturday,2021-04-03T17:24:00.000,101862106242,1018621,210206975,210932259,...,4,6,27,1,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

## Feature Selection

res = res.rename(columns = {'incident_datetime' : 'Incident Datetime', 'incident_date' : 'Incident Date',
                                          'incident_time' : 'Incident Time', 'incident_year' : 'Incident Year',
                                          'incident_day_of_week' : 'Incident Day of Week', 
                                          'report_datetime' : 'Report Datetime', 'row_id' : 'Row ID', 
                                          'incident_id' : 'Incident ID',
                                          'incident_number' : 'Incident Number', 
                                          'cad_number' : 'CAD Number', 
                                          'report_type_code' : 'Report Type Code',
                                          'report_type_description' : 'Report Type Description', 
                                          'incident_code' : 'Incident Code',
                                          'incident_category' : 'Incident Category', 
                                          'incident_subcategory' : 'Incident Subcategory', 
                                          'incident_description' : 'Incident Description',
                                          'resolution' : 'Resolution', 'intersection' : 'Intersection',
                                          'cnn' : 'CNN',
                                          'analysis_neighborhood' : 'Analysis Neighborhood', 
                                          'latitude' : 'Latitude', 'longitude' : 'Longitude',
                                          'point' : 'Point'})

## Spelling Normalization

In [ ]:
def clean_incident_category(df) : 
    df['Incident Category'].replace('Offence', 'Offense', regex = True, inplace = True)
    df['Incident Category'].replace('Offenses','Offense',regex=True, inplace = True)
    #df['Incident Category'].replace('Offense Against The Family And Children', 'Family Offense', regex=False, inplace = True)
    df['Incident Category'].replace('Human Trafficking (A), Commercial Sex Acts', 'Human Trafficking', regex=False, inplace = True)
    df['Incident Category'].replace('Human Trafficking, Commercial Sex Acts', 'Human Trafficking', regex=False, inplace = True)
    df['Incident Category'].replace('Human Trafficking (B), Involuntary Servitude', 'Human Trafficking', regex=False, inplace = True)
    df['Incident Category'].replace('Motor Vehicle Theft?', 'Motor Vehicle Theft', regex=False, inplace = True)
    df['Incident Category'].replace('Suspicious Occ', 'Suspicious', regex=False, inplace = True)
    return

In [ ]:
clean_incident_category(res)

In [ ]:
res['Incident Category'].value_counts()

Larceny Theft                              134614
Other Miscellaneous                         33011
Malicious Mischief                          28772
Non-Criminal                                27256
Assault                                     26586
Burglary                                    24736
Motor Vehicle Theft                         20252
Recovered Vehicle                           15845
Warrant                                     14935
Lost Property                               14141
Fraud                                       13417
Drug Offense                                10997
Robbery                                     10521
Missing Person                              10053
Suspicious                                   8843
Disorderly Conduct                           7608
Offense Against The Family And Children      6145
Traffic Violation Arrest                     5347
Miscellaneous Investigation                  4160
Other Offense                                3796


In [ ]:
res_= res.set_index('Incident ID')

## Interpreting NaN values
### CAD Number
The Computer Aided Dispatch (CAD) is the system used by the Department of Emergency Management (DEM) to dispatch officers and other public safety personnel. CAD Numbers are assigned by the DEM system and linked to relevant incident reports (Incident Number). Not all Incidents will have a CAD Number. Those filed online via Coplogic (refer to “Filed Online” field) and others not filed through the DEM system will not have CAD Numbers.

In [ ]:
res_cad = res_.copy()
res_cad['CAD Number'] = res_['CAD Number'].fillna(0)

In [ ]:
res_nona = res_cad.copy()
res_nona= res_nona.dropna()

In [ ]:
res_nona

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident Number,CAD Number,Report Type Code,...,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Analysis Neighborhood,Latitude,Longitude,Point
Incident ID,,,,,,,,,,,,,,,,,,,,,
618930,2018-01-01T09:26:00.000,2018-01-01T00:00:00.000,09:26,2018,Monday,2018-01-01T09:27:00.000,61893007041,171052174,173641140,VS,...,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,03RD ST \ HOLLISTER AVE,20471000,Bayview Hunters Point,37.72171587946975,-122.39594382884452,"{'latitude': '37.72171587946975', 'longitude':..."
618931,2018-01-01T02:30:00.000,2018-01-01T00:00:00.000,02:30,2018,Monday,2018-01-01T08:21:00.000,61893105041,180000768,180010668,II,...,Burglary,Burglary - Residential,"Burglary, Residence, Forcible Entry",Open or Active,LISBON ST \ PERSIA AVE,21719000,Excelsior,37.722000219874225,-122.43360633930074,"{'latitude': '37.722000219874225', 'longitude'..."
618932,2018-01-01T10:00:00.000,2018-01-01T00:00:00.000,10:00,2018,Monday,2018-01-01T10:20:00.000,61893275000,180000605,180010893,IS,...,Missing Person,Missing Person,Found Person,Open or Active,VAN NESS AVE \ WILLOW ST,25189000,Tenderloin,37.78337048750076,-122.42083185184009,"{'latitude': '37.78337048750076', 'longitude':..."
618935,2018-01-01T10:03:00.000,2018-01-01T00:00:00.000,10:03,2018,Monday,2018-01-01T10:04:00.000,61893565015,180000887,180011579,II,...,Other Miscellaneous,Other,"Driving, No License Issued",Cite or Arrest Adult,BRAZIL AVE \ MISSION ST,21769000,Outer Mission,37.72468255342173,-122.43479841474401,"{'latitude': '37.72468255342173', 'longitude':..."
618936,2018-01-01T09:01:00.000,2018-01-01T00:00:00.000,09:01,2018,Monday,2018-01-01T09:39:00.000,61893607041,171052958,180011403,VS,...,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,CUSTOM HOUSE PL \ JACKSON ST,24709000,Chinatown,37.796698028315056,-122.40129440446798,"{'latitude': '37.796698028315056', 'longitude'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018513,2021-03-30T15:30:00.000,2021-03-30T00:00:00.000,15:30,2021,Tuesday,2021-04-03T06:30:00.000,101851306242,210184224,210831619,IS,...,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, $50-$200",Open or Active,36TH AVE \ JOHN F KENNEDY DR,32850000,Golden Gate Park,37.76996700325964,-122.49526346636263,"{'latitude': '37.76996700325964', 'longitude':..."
1018606,2021-04-03T13:25:00.000,2021-04-03T00:00:00.000,13:25,2021,Saturday,2021-04-03T16:30:00.000,101860606244,210206856,210932079,II,...,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,COLUMBUS AVE \ BEACH ST,25598000,Russian Hill,37.806780111468534,-122.4195772441978,"{'latitude': '37.806780111468534', 'longitude'..."
1018613,2021-04-03T13:58:00.000,2021-04-03T00:00:00.000,13:58,2021,Saturday,2021-04-03T14:11:00.000,101861304014,210206492,210931580,II,...,Assault,Aggravated Assault,"Assault, Aggravated, W/ Force",Open or Active,CLAY ST \ LARKIN ST,25297000,Russian Hill,37.79259308227565,-122.41940178531726,"{'latitude': '37.79259308227565', 'longitude':..."


In [ ]:
res_nona.keys()

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident Number',
       'CAD Number', 'Report Type Code', 'Report Type Description',
       'Incident Code', 'Incident Category', 'Incident Subcategory',
       'Incident Description', 'Resolution', 'Intersection', 'CNN',
       'Analysis Neighborhood', 'Latitude', 'Longitude', 'Point'],
      dtype='object')

### Writing the CSV file

In [ ]:
res_nona.to_csv('clean_data.csv')

https://docs.google.com/document/d/1cORUVRe03mA1UtYBlmu6IGJ9N0C_9iZobGFMam2Hb_I/edit#
<br>
https://docs.google.com/document/d/18Lek20dH3E03ISEYtzWzkPqJENrNAdmSSCzjnzj50b4/edit